# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 3 2023 11:09AM,256613,19,ADV80011379,"AdvaGen Pharma, Ltd",Released
1,Feb 3 2023 10:41AM,256610,19,ADV80011390,"AdvaGen Pharma, Ltd",Released
2,Feb 3 2023 10:41AM,256610,19,ADV80011391,"AdvaGen Pharma, Ltd",Released
3,Feb 3 2023 10:41AM,256610,19,ADV80011393,"AdvaGen Pharma, Ltd",Released
4,Feb 3 2023 10:41AM,256610,19,ADV80011394,"AdvaGen Pharma, Ltd",Released
5,Feb 3 2023 10:41AM,256610,19,ADV80011395,"AdvaGen Pharma, Ltd",Released
6,Feb 3 2023 10:41AM,256610,19,ADV80011396,"AdvaGen Pharma, Ltd",Released
7,Feb 3 2023 10:41AM,256610,19,ADV80011397,"AdvaGen Pharma, Ltd",Released
8,Feb 3 2023 10:41AM,256610,19,ADV80011398,"AdvaGen Pharma, Ltd",Released
9,Feb 3 2023 10:41AM,256610,19,ADV80011370,"AdvaGen Pharma, Ltd",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
31,256606,Released,3
32,256607,Released,1
33,256609,Released,16
34,256610,Released,17
35,256613,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
256606,1.0,3.0
256607,NaN,1.0
256609,NaN,16.0
256610,NaN,17.0
256613,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
256513,1.0,0.0
256515,1.0,0.0
256531,0.0,1.0
256537,0.0,1.0
256545,1.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
256513,1,0
256515,1,0
256531,0,1
256537,0,1
256545,1,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,256513,1,0
1,256515,1,0
2,256531,0,1
3,256537,0,1
4,256545,1,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,256513,1,
1,256515,1,
2,256531,,1
3,256537,,1
4,256545,1,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd"
1,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd"
18,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.
34,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc."
35,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation
39,Feb 3 2023 10:16AM,256605,10,ISDIN Corporation
94,Feb 3 2023 10:10AM,256604,15,"Person & Covey, Inc."
99,Feb 3 2023 10:10AM,256603,15,"Person & Covey, Inc."
115,Feb 3 2023 10:02AM,256601,21,"NBTY Global, Inc."
116,Feb 3 2023 10:01AM,256600,21,"NBTY Global, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",,1
1,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",,17
2,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,,16
3,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",,1
4,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation,1,3
5,Feb 3 2023 10:16AM,256605,10,ISDIN Corporation,,55
6,Feb 3 2023 10:10AM,256604,15,"Person & Covey, Inc.",1,4
7,Feb 3 2023 10:10AM,256603,15,"Person & Covey, Inc.",6,10
8,Feb 3 2023 10:02AM,256601,21,"NBTY Global, Inc.",,1
9,Feb 3 2023 10:01AM,256600,21,"NBTY Global, Inc.",,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1,
1,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17,
2,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16,
3,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1,
4,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation,3,1
5,Feb 3 2023 10:16AM,256605,10,ISDIN Corporation,55,
6,Feb 3 2023 10:10AM,256604,15,"Person & Covey, Inc.",4,1
7,Feb 3 2023 10:10AM,256603,15,"Person & Covey, Inc.",10,6
8,Feb 3 2023 10:02AM,256601,21,"NBTY Global, Inc.",1,
9,Feb 3 2023 10:01AM,256600,21,"NBTY Global, Inc.",1,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1,
1,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17,
2,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16,
3,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1,
4,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation,3,1


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1.0,NaN
1,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17.0,NaN
2,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16.0,NaN
3,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1.0,NaN
4,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation,3.0,1.0


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 3 2023 11:09AM,256613,19,"AdvaGen Pharma, Ltd",1.0,0.0
1,Feb 3 2023 10:41AM,256610,19,"AdvaGen Pharma, Ltd",17.0,0.0
2,Feb 3 2023 10:40AM,256609,10,Eywa Pharma Inc.,16.0,0.0
3,Feb 3 2023 10:26AM,256607,22,"NBTY Global, Inc.",1.0,0.0
4,Feb 3 2023 10:17AM,256606,10,ISDIN Corporation,3.0,1.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,2565885,175.0,2.0
12,513028,0.0,2.0
15,1539587,48.0,20.0
16,513131,1.0,1.0
19,769806,22.0,11.0
21,1282868,5.0,0.0
22,513187,2.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,2565885,175.0,2.0
1,12,513028,0.0,2.0
2,15,1539587,48.0,20.0
3,16,513131,1.0,1.0
4,19,769806,22.0,11.0
5,21,1282868,5.0,0.0
6,22,513187,2.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,175.0,2.0
1,12,0.0,2.0
2,15,48.0,20.0
3,16,1.0,1.0
4,19,22.0,11.0
5,21,5.0,0.0
6,22,2.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,175.0
1,12,Released,0.0
2,15,Released,48.0
3,16,Released,1.0
4,19,Released,22.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,16,19,21,22
Status,,,,,,,
Executing,2.0,2.0,20.0,1.0,11.0,0.0,0.0
Released,175.0,0.0,48.0,1.0,22.0,5.0,2.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,16,19,21,22
0,Executing,2.0,2.0,20.0,1.0,11.0,0.0,0.0
1,Released,175.0,0.0,48.0,1.0,22.0,5.0,2.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,16,19,21,22
0,Executing,2.0,2.0,20.0,1.0,11.0,0.0,0.0
1,Released,175.0,0.0,48.0,1.0,22.0,5.0,2.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()